# Preprocessing Script

In [1]:
import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
import scipy.io as sio
from scipy import signal
from scipy.signal import spectrogram, hann, butter, filtfilt, freqz
from scipy import stats

import pickle
import json
import csv

#mne
import mne_bids
import mne
from mne.time_frequency import tfr_morlet 
from mne.stats import permutation_cluster_test
from mne.preprocessing import ICA, create_ecg_epochs


Import PyPerceive and the local functions of this repository

In [43]:
jennifer_user_path = os.getcwd()
while jennifer_user_path[-14:] != 'jenniferbehnke':
    jennifer_user_path = os.path.dirname(jennifer_user_path)

# directory to this Repository
project_path = os.path.join(jennifer_user_path, 'code', 'Beta_profile_project', 'Beta_profile')
sys.path.append(project_path)

os.chdir(project_path)

import src.beta.utils.find_folders as find_folders
importlib.reload(find_folders)

# import PyPerceive
project_path = find_folders.chdir_repository("Py_Perceive")

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)

import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as PyPerceive_find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers



# import all functions from the local repository
project_path = find_folders.chdir_repository("Beta_profile")

# utils
import src.beta.utils.find_folders as find_folders
import src.beta.utils.io as io

# tfr
import src.beta.preprocessing.tfr as tfr


importlib.reload(find_folders)
importlib.reload(io)
importlib.reload(tfr)

<module 'src.beta.preprocessing.tfr' from '/Users/jenniferbehnke/code/Beta_profile_project/Beta_profile/src/beta/preprocessing/tfr.py'>

Load MNE Objects from PyPerceive

In [16]:
loaded_data = io.load_py_perceive_object(
    sub="024",
    session="fu12m",
    condition="m0s0",
    hemisphere="Right"
)
loaded_data


	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

PerceiveData(sub='024', incl_modalities=['survey'], incl_session=['fu12m'], incl_condition=['m0s0'], incl_task=['rest'], incl_contact=['RingR', 'SegmIntraR', 'SegmInterR'], import_json=False, warn_for_metaNaNs=True, use_chronic_json_file=True, allow_NaNs_in_metadata=False)

In [17]:
loaded_data.survey.fu12m.m0s0.rest.RingR.run1.data.get_data()

array([[ -7.41379038, -10.89602526,  -9.2110729 , ...,  -1.01097142,
         -3.03291425,  -2.92058409],
       [ -2.04939002,  -5.23733005,  -2.61866502, ...,   1.70782502,
          2.16324502,   3.07408503],
       [ -3.59456503,  -5.27951739,  -5.61650786, ...,   2.69592377,
          1.79728252,  -1.01097142],
       [  1.36010569,   2.15350067,   1.70013211, ...,   5.10039633,
          4.87371205,   5.55376489],
       [ -4.55420004,  -4.66805504,  -7.51443007, ...,  -3.41565003,
         -5.57889505,  -7.40057507],
       [ -1.34796189,  -4.60553645,  -6.40281897, ...,  -2.02194283,
         -2.92058409,  -3.03291425]])

In [33]:
data = io.extract_data_from_py_perceive(
    sub="024",
    session="fu12m",
    condition="m0s0",
    hemisphere="Right"
)
data


	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

{'RingR': <RawArray | 6 x 5288 (21.1 s), ~258 kB, data loaded>,
 'SegmIntraR': <RawArray | 6 x 5288 (21.1 s), ~258 kB, data loaded>,
 'SegmInterR': <RawArray | 3 x 5188 (20.7 s), ~129 kB, data loaded>}

In [38]:
ring_data = data["SegmIntraR"]

In [44]:
structured_signals = tfr.pick_channels_of_interest(
    sub="024",
    session="fu12m",
    condition="m0s0",
    hemisphere="Right"
)


	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

In [51]:
structured_signals

,channel,signal,lfp_group
0,LFP_R_12_STN_MT,"[1.3601056882, 2.153500673, 1.7001321103, 2.72...",RingR
1,LFP_R_01_STN_MT,"[-4.554200042, -4.668055043, -7.5144300693, -7...",RingR
2,LFP_R_23_STN_MT,"[-1.3479618874, -4.6055364487, -6.4028189653, ...",RingR
3,LFP_R_1A1B_STN_MT,"[3.4822348759, 4.2685459769, 7.076799909, 7.63...",SegmIntraR
4,LFP_R_1B1C_STN_MT,"[-3.6433600336, -2.0493900189, -4.0987800378, ...",SegmIntraR
5,LFP_R_1A1C_STN_MT,"[1.2356317301, 2.358933303, 3.5945650331, 3.14...",SegmIntraR
6,LFP_R_2A2B_STN_MT,"[0.5667107034, 0.6800528441, -1.2467635475, -2...",SegmIntraR
7,LFP_R_2B2C_STN_MT,"[-3.3017950304, -2.6186650241, -2.5048100231, ...",SegmIntraR
8,LFP_R_2A2C_STN_MT,"[-4.4932062914, -3.8192253477, -3.145244404, -...",SegmIntraR
9,LFP_R_1A2A_STN_MT,"[-25.8359361757, -27.2962282204, -20.444088626...",SegmInterR
